In [3]:
import mne
import os.path as op
from tools import files
import numpy as np
import matplotlib.pylab as plt
from scipy.stats import trim_mean, ttest_1samp, sem
%matplotlib qt

In [4]:
# functions
trim = lambda x: trim_mean(x, 0.1, axis=0)

In [5]:
main_path = op.join("/cubric", "scratch", "c1557187", "act_mis")
meg_path = op.join(main_path, "MEG")
fs_path = "/cubric/scratch/c1557187/MRI_337/FS_OUTPUT"

In [6]:
participants = files.get_folders_files(meg_path, wp=False)[0]
participants.sort()
participant = participants[0]
pp_path = op.join(meg_path, participant, "new_v1")

In [7]:
results_path = op.join(main_path, "RESULTS", "WHATEVER_SOURCE_SPACE")
files.make_folder(results_path)

In [8]:
all_epochs = files.get_files(pp_path, "epochs-TD", "-epo.fif")[2]
all_epochs.sort()
fwd_solution = files.get_files(pp_path, "", "-fwd.fif")[2]
fwd_solution.sort()
noise_cov = files.get_files(pp_path, "epochs-TD", "-cov.fif")[2]
noise_cov.sort()

In [9]:
ix = 0

In [10]:
epochs = mne.read_epochs(all_epochs[ix])
cov = mne.read_cov(noise_cov[ix])
fwd = mne.read_forward_solution(fwd_solution[ix])

In [12]:
reg_avg = epochs["30"].average(method=trim)
odd_avg = epochs["40"].average(method=trim)

In [13]:
inverse_operator = mne.minimum_norm.make_inverse_operator(
    epochs.info,
    fwd,
    cov,
    loose=0.2,
    depth=0.8
)

In [14]:
diff = mne.combine_evoked([odd_avg, -reg_avg], weights=[30/140, 110/140])

In [15]:
regular_epochs = mne.minimum_norm.apply_inverse(
    diff,
    inverse_operator,
    lambda2= 1/3**2,
    method="dSPM"
)

In [16]:
surfer_kwargs = dict(
    hemi="both",
    subjects_dir=fs_path,
    subject=participant,
    views='lateral',
    time_unit="s",
    size=(800, 800),
    smoothing_steps=5,
    time_viewer=True
)

In [ ]:
brain = regular_epochs.plot(**surfer_kwargs)

In [ ]:
brain.add_foci(vert_n, coords_as_verts=True, hemi='lh', color='blue',
               scale_factor=0.6, alpha=0.5)

In [13]:
src = mne.setup_source_space(
    subject="fsaverage",
    subjects_dir=fs_path,
    spacing="ico4",
    add_dist=False,
    n_jobs=2
)

In [14]:
src[0]["subject_his_id"]= "0001"
src[1]["subject_his_id"]= "0001"


In [17]:
morph = mne.compute_source_morph(
    regular_epochs, 
    subject_from=participant,
    subject_to="fsaverage",
    subjects_dir=fs_path
)

In [19]:
diff_fsaverage = morph.apply(regular_epochs)

In [20]:
surfer_kwargs = dict(
    hemi="both",
    subjects_dir=fs_path,
    subject="fsaverage",
    views='lateral',
    time_unit="s",
    size=(800, 800),
    smoothing_steps=5,
    time_viewer=True
)

In [ ]:
diff_fsaverage.plot(**surfer_kwargs);

In [ ]:
s